Идея состоит в том, что надо показать как файл из S3-хранилища сам перейдет в колоночную базу с SQL-интерфейсом.

Ещё надо показать как можно работать с ним через http-интерфейс.



In [1]:
import pika
import threading
import time

In [2]:
con_par = pika.ConnectionParameters('127.0.0.1', 5672)
connection1 = pika.BlockingConnection(con_par)
channel1 = connection1.channel()
connection2 = pika.BlockingConnection(con_par)
channel2 = connection2.channel()

In [3]:
channel1.queue_declare(queue='hello')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=hello'])>"])>

In [4]:
channel1.basic_publish(exchange='',
                       routing_key='hello',
                       body='Hello World!')

In [5]:
def generate_messages():
    for i in range(10):
        channel1.basic_publish(exchange='',
                          routing_key='hello',
                          body='Hello World!')
        print("next one")
        time.sleep(1)
    
thr = threading.Thread(target=generate_messages)
thr.start()

next one
next one


In [6]:
messages = 0

def callback(ch, method, properties, body):
    global messages
    messages += 1
    print(" [x] Received %r" % body)
    
def consume_messages(channel):
    channel.basic_consume(queue='hello',
                          auto_ack=True,
                          on_message_callback=callback)
    channel.start_consuming()
    
thr1 = threading.Thread(target=consume_messages, args=([channel2]))
thr1.start()

 [x] Received b'Hello World!'
 [x] Received b'Hello World!'
 [x] Received b'Hello World!'
next one
 [x] Received b'Hello World!'
next one
 [x] Received b'Hello World!'
next one
 [x] Received b'Hello World!'
next one
 [x] Received b'Hello World!'
next one
 [x] Received b'Hello World!'
next one
 [x] Received b'Hello World!'


In [6]:
method_frame, header_frame, body = channel2.basic_get('hello')
print(body)

b'Hello World!'


In [7]:
connection1.close()
connection2.close()


In [2]:
import requests
import re
from bs4 import BeautifulSoup
import datetime
import time
import io
from minio import Minio

In [47]:
minioClient = Minio('localhost:9000',
                  access_key='12345678',
                  secret_key='12345678', secure=False)

con_par_writer = pika.ConnectionParameters('127.0.0.1', 5672)
connection_writer = pika.BlockingConnection(con_par_writer)
channel_writer = connection_writer.channel()

In [48]:
channel_writer.queue_declare(queue='lentaq')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=lentaq'])>"])>

In [49]:
# Функция для загрузки одной новости из Ленты.ру
def getLentaArticle(url):
    """ getLentaArticle gets the body of the article from Lenta.ru"""
    print("downloading lenta:", url)
    r = requests.get(url)
    p = BeautifulSoup(r.text, "html5")("p")
    if len(p) == 0:
        return "  "
    p = " ".join([a.get_text() for a in p])
    return p

# Загрузка новостей из Ленты.ру за некоторый период.
def getLenta():
    curdate = datetime.date(2017, 1, 16)
    findate = datetime.date(2017, 1, 16)
    res = ""
    
    fileno = 1
# Загружаем новости до конечной даты.
    while curdate <= findate:
        print('lenta ' + curdate.strftime('%Y/%m/%d'))
        day = requests.get('https://lenta.ru/news/' + curdate.strftime('%Y/%m/%d'))
        body = re.findall('<h3>(.+?)</h3>', day.text)
        links = ['https://lenta.ru' + re.findall('"(.+?)"', x)[0] for x in body]
        for l in links: # идем по всем ссылкам на новости за день.
            news = getLentaArticle(l)
            if news != "  ":
                # Преобразуем текст к набору байт.
                news = news.encode()
                news_len = len(news)
                f = io.BytesIO(news)
                # Кладем их в Minio.
                filename = f"lenta{curdate.strftime('%Y_%m_%d')}_{fileno}.txt"
                minioClient.put_object('lenta', filename, f, news_len, content_type='text/plain')
                fileno += 1
                # Сообщение добавляем в RabbitMQ.
                channel_writer.basic_publish(exchange='', routing_key='lentaq', body=filename)
                if fileno>10:
                    break
            time.sleep(0.2)
        curdate += datetime.timedelta(days=1)
    print("lenta finished")


In [50]:
getLenta()

lenta 2017/01/16
downloading lenta: https://lenta.ru/news/2017/01/16/apollo/
downloading lenta: https://lenta.ru/news/2017/01/16/kadyrov/
downloading lenta: https://lenta.ru/news/2017/01/16/trump2/
downloading lenta: https://lenta.ru/news/2017/01/16/accordion/
downloading lenta: https://lenta.ru/news/2017/01/16/trump3/
downloading lenta: https://lenta.ru/news/2017/01/16/bishkek4/
downloading lenta: https://lenta.ru/news/2017/01/16/bishkek5/
downloading lenta: https://lenta.ru/news/2017/01/16/realsevilla/
downloading lenta: https://lenta.ru/news/2017/01/16/safetymoscow/
downloading lenta: https://lenta.ru/news/2017/01/16/vertols/
lenta finished


In [51]:
def consume_lenta(ch, method, properties, body):
    print(body)
    file_obj = minioClient2.get_object('lenta', body)
    print(file_obj.data.decode()[:100])
    

con_par_consume = pika.ConnectionParameters('127.0.0.1', 5672)
connection_consume = pika.BlockingConnection(con_par_consume)
channel_consume = connection_consume.channel()
channel_consume.basic_consume(queue='lentaq',
                              auto_ack=True,
                              on_message_callback=consume_lenta)

minioClient2 = Minio('localhost:9000',
                     access_key='12345678',
                     secret_key='12345678', secure=False)


In [54]:
try:
    channel_consume.start_consuming()
except KeyboardInterrupt:
    channel_consume.stop_consuming()


In [53]:
channel_consume.basic_get('lentaq')

(None, None, None)